In [139]:
import pandas as pd
import numpy as np
import plotly.express as px

In [140]:
import plotly.io as pio

pio.renderers.default = "vscode"  # or "vscode" if using VS Code notebooks

template = pio.templates["plotly_dark"]

# Backgrounds
template.layout.plot_bgcolor = "#2B2B2B"   # inside the axes
template.layout.paper_bgcolor = "#2B2B2B"  # around the plot

# X axis
template.layout.xaxis.color = "#A9B7C6"    # tick labels + title
template.layout.xaxis.gridcolor = "#7B7E82"
template.layout.xaxis.showline = False
template.layout.xaxis.linecolor = "#A9B7C6"
template.layout.xaxis.tickcolor = "#A9B7C6"  # color of tick lines
template.layout.xaxis.zeroline = False
template.layout.xaxis.zerolinecolor = "#A9B7C6"

# Y axis
template.layout.yaxis.color = "#A9B7C6"
template.layout.yaxis.gridcolor = "#7B7E82"
template.layout.yaxis.showline = False
template.layout.yaxis.linecolor = "#A9B7C6"
template.layout.yaxis.tickcolor = "#A9B7C6"
template.layout.yaxis.zeroline = False
template.layout.yaxis.zerolinecolor = "#A9B7C6"

# Register as new template
pio.templates["custom_dark"] = template
pio.templates.default = "custom_dark"

### **1. Load data and ensure correct datetime**

In [141]:
df = pd.read_csv(
    "processed/cleaned_OHLCV_730d_1hr.csv",
    parse_dates=["timestamp"],
)
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
df["timestamp"] = df["timestamp"].dt.tz_convert("America/New_York")
df

,timestamp,ticker,close,high,low,open,volume
0,2022-12-27 09:30:00-05:00,AAPL,130.165207,131.410004,128.720001,131.380005,19654999.0
1,2022-12-27 10:30:00-05:00,AAPL,129.725693,130.389999,129.179993,130.160004,10795118.0
2,2022-12-27 11:30:00-05:00,AAPL,129.929993,130.529907,129.509995,129.720001,7518424.0
3,2022-12-27 12:30:00-05:00,AAPL,130.005005,130.369995,129.679993,129.925003,5515019.0
4,2022-12-27 13:30:00-05:00,AAPL,129.929901,130.160004,129.619995,129.994995,6673932.0
...,...,...,...,...,...,...,...
35765,2025-11-21 11:30:00-05:00,TSLA,400.375000,401.209991,389.748199,391.470001,12205100.0
35766,2025-11-21 12:30:00-05:00,TSLA,394.640015,402.250000,393.170197,400.390015,8240202.0
35767,2025-11-21 13:30:00-05:00,TSLA,401.700012,402.320007,394.380005,394.640015,8102757.0
35768,2025-11-21 14:30:00-05:00,TSLA,395.790497,402.010010,395.709991,401.739990,6210947.0


### **2. Returns**

In [142]:
ticker = df.groupby("ticker")
df["ret"] = ticker["close"].pct_change()
df["log_ret"] = ticker["close"].apply(lambda x: np.log(x / x.shift(1))).reset_index(level=0, drop=True)
df

,timestamp,ticker,close,high,low,open,volume,ret,log_ret
0,2022-12-27 09:30:00-05:00,AAPL,130.165207,131.410004,128.720001,131.380005,19654999.0,NaN,NaN
1,2022-12-27 10:30:00-05:00,AAPL,129.725693,130.389999,129.179993,130.160004,10795118.0,-0.003377,-0.003382
2,2022-12-27 11:30:00-05:00,AAPL,129.929993,130.529907,129.509995,129.720001,7518424.0,0.001575,0.001574
3,2022-12-27 12:30:00-05:00,AAPL,130.005005,130.369995,129.679993,129.925003,5515019.0,0.000577,0.000577
4,2022-12-27 13:30:00-05:00,AAPL,129.929901,130.160004,129.619995,129.994995,6673932.0,-0.000578,-0.000578
...,...,...,...,...,...,...,...,...,...
35765,2025-11-21 11:30:00-05:00,TSLA,400.375000,401.209991,389.748199,391.470001,12205100.0,0.022983,0.022722
35766,2025-11-21 12:30:00-05:00,TSLA,394.640015,402.250000,393.170197,400.390015,8240202.0,-0.014324,-0.014428
35767,2025-11-21 13:30:00-05:00,TSLA,401.700012,402.320007,394.380005,394.640015,8102757.0,0.017890,0.017732
35768,2025-11-21 14:30:00-05:00,TSLA,395.790497,402.010010,395.709991,401.739990,6210947.0,-0.014711,-0.014821


### **3. Rolling volatility**
- try different windows
- remember that we need to do in terms of trading days so would be 24, 48, 72, etc... hours

In [143]:
VOL_WINDOWS = [24, 48, 72]  # in hours
for window in VOL_WINDOWS:
    df[f"roll_vol_{window}_hr"] = ticker["log_ret"].rolling(window).std().reset_index(level=0, drop=True)
df

,timestamp,ticker,close,high,low,open,volume,ret,log_ret,roll_vol_24_hr,roll_vol_48_hr,roll_vol_72_hr
0,2022-12-27 09:30:00-05:00,AAPL,130.165207,131.410004,128.720001,131.380005,19654999.0,NaN,NaN,NaN,NaN,NaN
1,2022-12-27 10:30:00-05:00,AAPL,129.725693,130.389999,129.179993,130.160004,10795118.0,-0.003377,-0.003382,NaN,NaN,NaN
2,2022-12-27 11:30:00-05:00,AAPL,129.929993,130.529907,129.509995,129.720001,7518424.0,0.001575,0.001574,NaN,NaN,NaN
3,2022-12-27 12:30:00-05:00,AAPL,130.005005,130.369995,129.679993,129.925003,5515019.0,0.000577,0.000577,NaN,NaN,NaN
4,2022-12-27 13:30:00-05:00,AAPL,129.929901,130.160004,129.619995,129.994995,6673932.0,-0.000578,-0.000578,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
35765,2025-11-21 11:30:00-05:00,TSLA,400.375000,401.209991,389.748199,391.470001,12205100.0,0.022983,0.022722,0.019028,0.016545,0.014689
35766,2025-11-21 12:30:00-05:00,TSLA,394.640015,402.250000,393.170197,400.390015,8240202.0,-0.014324,-0.014428,0.018218,0.016649,0.014696
35767,2025-11-21 13:30:00-05:00,TSLA,401.700012,402.320007,394.380005,394.640015,8102757.0,0.017890,0.017732,0.018189,0.016878,0.014861
35768,2025-11-21 14:30:00-05:00,TSLA,395.790497,402.010010,395.709991,401.739990,6210947.0,-0.014711,-0.014821,0.018410,0.016988,0.014945


### **4. Volume z-score**

In [144]:
VOLUME_Z_WINDOW = 96  # ~4 trading days
volume_mean = ticker["volume"].rolling(VOLUME_Z_WINDOW).mean().reset_index(level=0, drop=True)
volume_std  = ticker["volume"].rolling(VOLUME_Z_WINDOW).std().reset_index(level=0, drop=True)

df[f"volume_mean_{VOLUME_Z_WINDOW}_hr"] = volume_mean
df[f"volume_std_{VOLUME_Z_WINDOW}_hr"]  = volume_std
df[f"volume_z_{VOLUME_Z_WINDOW}_hr"]    = (df["volume"] - df[f"volume_mean_{VOLUME_Z_WINDOW}_hr"]) / df[f"volume_std_{VOLUME_Z_WINDOW}_hr"]
df

,timestamp,ticker,close,high,low,open,volume,ret,log_ret,roll_vol_24_hr,roll_vol_48_hr,roll_vol_72_hr,volume_mean_96_hr,volume_std_96_hr,volume_z_96_hr
0,2022-12-27 09:30:00-05:00,AAPL,130.165207,131.410004,128.720001,131.380005,19654999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-12-27 10:30:00-05:00,AAPL,129.725693,130.389999,129.179993,130.160004,10795118.0,-0.003377,-0.003382,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-12-27 11:30:00-05:00,AAPL,129.929993,130.529907,129.509995,129.720001,7518424.0,0.001575,0.001574,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-27 12:30:00-05:00,AAPL,130.005005,130.369995,129.679993,129.925003,5515019.0,0.000577,0.000577,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-12-27 13:30:00-05:00,AAPL,129.929901,130.160004,129.619995,129.994995,6673932.0,-0.000578,-0.000578,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35765,2025-11-21 11:30:00-05:00,TSLA,400.375000,401.209991,389.748199,391.470001,12205100.0,0.022983,0.022722,0.019028,0.016545,0.014689,9.482434e+06,5.190753e+06,0.524522
35766,2025-11-21 12:30:00-05:00,TSLA,394.640015,402.250000,393.170197,400.390015,8240202.0,-0.014324,-0.014428,0.018218,0.016649,0.014696,9.501033e+06,5.182983e+06,-0.243264
35767,2025-11-21 13:30:00-05:00,TSLA,401.700012,402.320007,394.380005,394.640015,8102757.0,0.017890,0.017732,0.018189,0.016878,0.014861,9.541270e+06,5.156637e+06,-0.278963
35768,2025-11-21 14:30:00-05:00,TSLA,395.790497,402.010010,395.709991,401.739990,6210947.0,-0.014711,-0.014821,0.018410,0.016988,0.014945,9.415539e+06,5.087988e+06,-0.629835


### **5. Recent Support level (rolling low of feature - low)**

In [153]:
SUPPORT_WINDOW = 48  # past 48 hours

g = df.groupby("ticker")

df["recent_low"] = (
    g["low"]
    .shift(1)                                # exclude current bar
    .rolling(SUPPORT_WINDOW)
    .min()
    .reset_index(level=0, drop=True)
)
df.tail(5)

,timestamp,ticker,close,high,low,open,volume,ret,log_ret,roll_vol_24_hr,...,volume_mean_96_hr,volume_std_96_hr,volume_z_96_hr,recent_low,dist_to_low,dist_to_low_pct,drawdown,fwd_ret_3h,is_cascade_candidate,label_mean_revert
35765,2025-11-21 11:30:00-05:00,TSLA,400.375000,401.209991,389.748199,391.470001,12205100.0,0.022983,0.022722,0.019028,...,9.482434e+06,5.190753e+06,0.524522,382.779999,17.595001,0.043946,-0.146904,-0.011451,False,0
35766,2025-11-21 12:30:00-05:00,TSLA,394.640015,402.250000,393.170197,400.390015,8240202.0,-0.014324,-0.014428,0.018218,...,9.501033e+06,5.182983e+06,-0.243264,382.779999,11.860016,0.030053,-0.157526,-0.009097,False,0
35767,2025-11-21 13:30:00-05:00,TSLA,401.700012,402.320007,394.380005,394.640015,8102757.0,0.017890,0.017732,0.018189,...,9.541270e+06,5.156637e+06,-0.278963,382.779999,18.920013,0.047100,-0.130990,NaN,False,0
35768,2025-11-21 14:30:00-05:00,TSLA,395.790497,402.010010,395.709991,401.739990,6210947.0,-0.014711,-0.014821,0.018410,...,9.415539e+06,5.087988e+06,-0.629835,382.779999,13.010498,0.032872,-0.143774,NaN,False,0
35769,2025-11-21 15:30:00-05:00,TSLA,391.049988,397.600006,390.559998,395.760010,5417647.0,-0.011977,-0.012050,0.018511,...,9.388876e+06,5.102291e+06,-0.778323,382.779999,8.269989,0.021148,-0.154029,NaN,False,0


### **6. Distance to recent low**

In [146]:
df["dist_to_low"] = df["close"] - df["recent_low"]
df["dist_to_low_pct"] = df["dist_to_low"] / df["close"]
df.tail(5)

,timestamp,ticker,close,high,low,open,volume,ret,log_ret,roll_vol_24_hr,roll_vol_48_hr,roll_vol_72_hr,volume_mean_96_hr,volume_std_96_hr,volume_z_96_hr,recent_low,dist_to_low,dist_to_low_pct
35765,2025-11-21 11:30:00-05:00,TSLA,400.375000,401.209991,389.748199,391.470001,12205100.0,0.022983,0.022722,0.019028,0.016545,0.014689,9.482434e+06,5.190753e+06,0.524522,382.779999,17.595001,0.043946
35766,2025-11-21 12:30:00-05:00,TSLA,394.640015,402.250000,393.170197,400.390015,8240202.0,-0.014324,-0.014428,0.018218,0.016649,0.014696,9.501033e+06,5.182983e+06,-0.243264,382.779999,11.860016,0.030053
35767,2025-11-21 13:30:00-05:00,TSLA,401.700012,402.320007,394.380005,394.640015,8102757.0,0.017890,0.017732,0.018189,0.016878,0.014861,9.541270e+06,5.156637e+06,-0.278963,382.779999,18.920013,0.047100
35768,2025-11-21 14:30:00-05:00,TSLA,395.790497,402.010010,395.709991,401.739990,6210947.0,-0.014711,-0.014821,0.018410,0.016988,0.014945,9.415539e+06,5.087988e+06,-0.629835,382.779999,13.010498,0.032872
35769,2025-11-21 15:30:00-05:00,TSLA,391.049988,397.600006,390.559998,395.760010,5417647.0,-0.011977,-0.012050,0.018511,0.015631,0.014975,9.388876e+06,5.102291e+06,-0.778323,382.779999,8.269989,0.021148


### **7. Simple drawdown relative to rolling max of close**

In [147]:
ROLL_MAX_WINDOW = 96
roll_max = ticker["close"].rolling(ROLL_MAX_WINDOW).max().reset_index(level=0, drop=True)
df["drawdown"] = df["close"] / roll_max - 1.0
df.tail(5)

,timestamp,ticker,close,high,low,open,volume,ret,log_ret,roll_vol_24_hr,roll_vol_48_hr,roll_vol_72_hr,volume_mean_96_hr,volume_std_96_hr,volume_z_96_hr,recent_low,dist_to_low,dist_to_low_pct,drawdown
35765,2025-11-21 11:30:00-05:00,TSLA,400.375000,401.209991,389.748199,391.470001,12205100.0,0.022983,0.022722,0.019028,0.016545,0.014689,9.482434e+06,5.190753e+06,0.524522,382.779999,17.595001,0.043946,-0.146904
35766,2025-11-21 12:30:00-05:00,TSLA,394.640015,402.250000,393.170197,400.390015,8240202.0,-0.014324,-0.014428,0.018218,0.016649,0.014696,9.501033e+06,5.182983e+06,-0.243264,382.779999,11.860016,0.030053,-0.157526
35767,2025-11-21 13:30:00-05:00,TSLA,401.700012,402.320007,394.380005,394.640015,8102757.0,0.017890,0.017732,0.018189,0.016878,0.014861,9.541270e+06,5.156637e+06,-0.278963,382.779999,18.920013,0.047100,-0.130990
35768,2025-11-21 14:30:00-05:00,TSLA,395.790497,402.010010,395.709991,401.739990,6210947.0,-0.014711,-0.014821,0.018410,0.016988,0.014945,9.415539e+06,5.087988e+06,-0.629835,382.779999,13.010498,0.032872,-0.143774
35769,2025-11-21 15:30:00-05:00,TSLA,391.049988,397.600006,390.559998,395.760010,5417647.0,-0.011977,-0.012050,0.018511,0.015631,0.014975,9.388876e+06,5.102291e+06,-0.778323,382.779999,8.269989,0.021148,-0.154029


### **8. Forward returns for labelling**

In [148]:
H_FWD = 3
H_FWD = 3

# Forward close price within each ticker
fwd_close = ticker["close"].shift(-H_FWD)

# Make sure it's a flat Series aligned to df
fwd_close = fwd_close.reset_index(level=0, drop=True)

# Compute forward return
df["fwd_ret_3h"] = fwd_close / df["close"] - 1.0
df

,timestamp,ticker,close,high,low,open,volume,ret,log_ret,roll_vol_24_hr,roll_vol_48_hr,roll_vol_72_hr,volume_mean_96_hr,volume_std_96_hr,volume_z_96_hr,recent_low,dist_to_low,dist_to_low_pct,drawdown,fwd_ret_3h
0,2022-12-27 09:30:00-05:00,AAPL,130.165207,131.410004,128.720001,131.380005,19654999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001231
1,2022-12-27 10:30:00-05:00,AAPL,129.725693,130.389999,129.179993,130.160004,10795118.0,-0.003377,-0.003382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001574
2,2022-12-27 11:30:00-05:00,AAPL,129.929993,130.529907,129.509995,129.720001,7518424.0,0.001575,0.001574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002540
3,2022-12-27 12:30:00-05:00,AAPL,130.005005,130.369995,129.679993,129.925003,5515019.0,0.000577,0.000577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000192
4,2022-12-27 13:30:00-05:00,AAPL,129.929901,130.160004,129.619995,129.994995,6673932.0,-0.000578,-0.000578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35765,2025-11-21 11:30:00-05:00,TSLA,400.375000,401.209991,389.748199,391.470001,12205100.0,0.022983,0.022722,0.019028,0.016545,0.014689,9.482434e+06,5.190753e+06,0.524522,382.779999,17.595001,0.043946,-0.146904,-0.011451
35766,2025-11-21 12:30:00-05:00,TSLA,394.640015,402.250000,393.170197,400.390015,8240202.0,-0.014324,-0.014428,0.018218,0.016649,0.014696,9.501033e+06,5.182983e+06,-0.243264,382.779999,11.860016,0.030053,-0.157526,-0.009097
35767,2025-11-21 13:30:00-05:00,TSLA,401.700012,402.320007,394.380005,394.640015,8102757.0,0.017890,0.017732,0.018189,0.016878,0.014861,9.541270e+06,5.156637e+06,-0.278963,382.779999,18.920013,0.047100,-0.130990,NaN
35768,2025-11-21 14:30:00-05:00,TSLA,395.790497,402.010010,395.709991,401.739990,6210947.0,-0.014711,-0.014821,0.018410,0.016988,0.014945,9.415539e+06,5.087988e+06,-0.629835,382.779999,13.010498,0.032872,-0.143774,NaN


In [156]:
df[["log_ret","roll_vol_48_hr","volume_z_96_hr","dist_to_low_pct"]].describe()

,log_ret,roll_vol_48_hr,volume_z_96_hr,dist_to_low_pct
count,35763.000000,35434.000000,35105.000000,35434.000000
mean,0.000231,0.007602,0.001276,0.046770
std,0.008958,0.004713,1.028259,0.042419
min,-0.139523,0.000959,-2.521331,-0.102223
25%,-0.002494,0.004335,-0.632512,0.017373
50%,0.000179,0.006384,-0.320896,0.034844
75%,0.002903,0.009682,0.280301,0.064492
max,0.227867,0.034951,9.242226,0.329761


### **9. Cascade candidate flag**

In [194]:
cond_ret  = df["log_ret"] < -0.005           # need -0.8% log returns
cond_vol  = df["volume_z_96_hr"] > 0.5       # volume z-score > 0.8 (spike in volume)
cond_supp = df["dist_to_low"] < 0            # close is below recent support

ts = pd.to_datetime(df["timestamp"])         # provisionally filter out open bars as they combine overnight changes
is_open_bar = (df["timestamp"].dt.hour == 9) & (df["timestamp"].dt.minute == 30)

print("ret condition:", cond_ret.mean())
print("vol condition:", cond_vol.mean())
print("support break:", cond_supp.mean())

df["is_cascade_candidate"] = cond_ret & cond_vol & cond_supp & (~is_open_bar)
df["is_cascade_candidate"].value_counts()

ret condition: 0.13997763488957227
vol condition: 0.20069890970086665
support break: 0.025999440872239307


is_cascade_candidate
False    35615
True       155
Name: count, dtype: int64

### **10. Mean reversion vs continuation label**

In [195]:
# Mean reversion if fwd_ret_3h > 0, continuation if < 0
df["label_mean_revert"] = (df["fwd_ret_3h"] > 0).astype(int)
df.tail(5)

,timestamp,ticker,close,high,low,open,volume,ret,log_ret,roll_vol_24_hr,...,volume_mean_96_hr,volume_std_96_hr,volume_z_96_hr,recent_low,dist_to_low,dist_to_low_pct,drawdown,fwd_ret_3h,is_cascade_candidate,label_mean_revert
35765,2025-11-21 11:30:00-05:00,TSLA,400.375000,401.209991,389.748199,391.470001,12205100.0,0.022983,0.022722,0.019028,...,9.482434e+06,5.190753e+06,0.524522,382.779999,17.595001,0.043946,-0.146904,-0.011451,False,0
35766,2025-11-21 12:30:00-05:00,TSLA,394.640015,402.250000,393.170197,400.390015,8240202.0,-0.014324,-0.014428,0.018218,...,9.501033e+06,5.182983e+06,-0.243264,382.779999,11.860016,0.030053,-0.157526,-0.009097,False,0
35767,2025-11-21 13:30:00-05:00,TSLA,401.700012,402.320007,394.380005,394.640015,8102757.0,0.017890,0.017732,0.018189,...,9.541270e+06,5.156637e+06,-0.278963,382.779999,18.920013,0.047100,-0.130990,NaN,False,0
35768,2025-11-21 14:30:00-05:00,TSLA,395.790497,402.010010,395.709991,401.739990,6210947.0,-0.014711,-0.014821,0.018410,...,9.415539e+06,5.087988e+06,-0.629835,382.779999,13.010498,0.032872,-0.143774,NaN,False,0
35769,2025-11-21 15:30:00-05:00,TSLA,391.049988,397.600006,390.559998,395.760010,5417647.0,-0.011977,-0.012050,0.018511,...,9.388876e+06,5.102291e+06,-0.778323,382.779999,8.269989,0.021148,-0.154029,NaN,False,0


In [183]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def plot_cascade_event_plotly(df, ticker, event_number=0, window_hours=24):
    """
    Plot a single cascade event for a given ticker using Plotly.

    df: DataFrame with columns:
        ['timestamp', 'ticker', 'close', 'recent_low', 'log_ret', 'volume', 'is_cascade_candidate']
    ticker: e.g. "AAPL"
    event_number: which cascade event to plot (0 = first, 1 = second, etc.)
    window_hours: number of hours before and after event to show
    """
    # Ensure timestamp is datetime
    if df["timestamp"].dtype == "O":
        df["timestamp"] = pd.to_datetime(df["timestamp"])

    # Filter for the chosen ticker
    df_t = df[df["ticker"] == ticker].copy()
    if df_t.empty:
        print(f"No data for ticker {ticker}")
        return

    # Set timestamp as index for time slicing
    df_t = df_t.sort_values("timestamp").set_index("timestamp")

    # Get all cascade event timestamps for this ticker
    cascade_times = df_t.index[df_t["is_cascade_candidate"]].to_list()
    if len(cascade_times) == 0:
        print(f"No cascade events found for {ticker}.")
        return

    # Clip event_number
    if event_number < 0 or event_number >= len(cascade_times):
        print(f"event_number {event_number} out of range (0 to {len(cascade_times)-1}).")
        return

    t0 = cascade_times[event_number]
    start = t0 - pd.Timedelta(hours=window_hours)
    end   = t0 + pd.Timedelta(hours=window_hours)

    df_win = df_t.loc[start:end]

    # Create 3-row subplot
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.5, 0.25, 0.25],
        subplot_titles=(
            f"{ticker} — Price & Support (Event at {t0})",
            "Log Returns",
            "Volume"
        )
    )

    # 1) Price + Support + Cascade marker
    fig.add_trace(
        go.Scatter(
            x=df_win.index,
            y=df_win["close"],
            mode="lines",
            name="Close",
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=df_win.index,
            y=df_win["recent_low"],
            mode="lines",
            name="Recent Low (Support)",
            line=dict(dash="dash")
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=[t0],
            y=[df_t.loc[t0, "close"]],
            mode="markers",
            name="Cascade Event",
            marker=dict(size=12, color="red", symbol="x"),
        ),
        row=1, col=1
    )

    # 2) Log returns (bars)
    fig.add_trace(
        go.Bar(
            x=df_win.index,
            y=df_win["log_ret"],
            name="Log Return",
        ),
        row=2, col=1
    )

    fig.add_hline(
        y=0,
        line_width=1,
        line_color="black",
        row=2, col=1
    )

    # 3) Volume (bars)
    fig.add_trace(
        go.Bar(
            x=df_win.index,
            y=df_win["volume"],
            name="Volume",
        ),
        row=3, col=1
    )

    fig.update_layout(
        height=800,
        showlegend=True,
        title_text=f"Cascade Event {event_number} for {ticker}",
        xaxis3_title="Time",
    )

    fig.show()

In [201]:
# First cascade event for AAPL, ±24 hours around the event
plot_cascade_event_plotly(df, ticker="SPY", event_number=16, window_hours=100)